# Import modules

In [1]:
import pandas as pd
from Constraints import *

# About Constraints.py

__Constraints.Constraint__<br />
Populate constraint objects with name, setback, constraint feature class, where clause, and constraint class ie Be Aware, Minimize, Avoid. ALso hold paths to intermediate data in constraints processing<br />

These will be used as input in ConstraintsModel below

In [2]:
Constraint?

__Init signature:__ Constraint(name, setback, feature_class, where_clause, constraint_class)<br />
__Docstring:__      Holds data related to the constraint including intermediary data paths like projected and clipped data.<br />
__File:__           s:\toolsandresources\scripts\constraintsanalysis\constraints.py<br />

__Constraints.ConstraintModel__<br />
Processes constraints uses a list containing Constraint objects. Creates intermediate gdbs for projected, clipped etc data. <br />

These will be used as input in ConstraintsModel below

In [3]:
ConstraintsModel?


__Init signature:__<br />
ConstraintsModel(
    sites_fc, 
    constraints, 
    output_folder, 
    output_epsg=3400, 
    overwrite_outputs=True
)<br /><br />
__Docstring:__     <br />
Processes constraints uses a list containing Constraint objects<br />
Creates intermediate gdbs for projected, clipped etc data<br />
__File:__           s:\toolsandresources\scripts\constraintsanalysis\constraints.py<br />

# Load constraints parameters from excel

These will be project specific

In [4]:
working_folder = r'.'

constraint_csv = r".\Example_Constraints_Processing_Parameters.csv"
setback_field_name = 'Setback'

constraints_df = pd.read_csv(constraint_csv)
constraints_df = constraints_df.fillna('')

constraints_df

,Category,ConstraintName,Setback,Constraint Class,Data Source,where
0,Infrastructure,Numbered Highway ROW,40.0,No-go,.\Constraints\Constraints.gdb\Highway_ROW,
1,Infrastructure,County Roads ROW,100.0,Mitigate,.\Constraints\Constraints.gdb\Road_ROW,
2,Infrastructure,Residence,1000.0,No-go,.\Constraints\Constraints.gdb\Receptors_Reside...,Status = 'Occupied'
3,Infrastructure,Parcel boundary,50.0,Mitigate,.\Constraints\Constraints.gdb\Property_Line,
4,Infrastructure,Zoning Issues,0.0,No-go,.\Constraints\Constraints.gdb\Zoning_Issues_Ap...,
5,Infrastructure,Communications Towers,125.0,Mitigate,.\Constraints\Constraints.gdb\Telecom_Tower,
6,Infrastructure,Overhead Distribution Lines,50.0,No-go,.\Constraints\Constraints.gdb\Electric_Distrib...,
7,Oil and Gas,Abandoned Oil And Gas Wells,5.0,Be Aware,.\Constraints\Constraints.gdb\Abandoned_Well,
8,Oil and Gas,Oil And Gas Wells,0.0,Mitigate,.\Constraints\Constraints.gdb\Well_Lease,
9,Oil and Gas,Pipelines ROW,0.0,Be Aware,.\Constraints\Constraints.gdb\Pipeline_ROW,


__Populate Constraint objects with data from excel__

In [5]:
constraints = []
for index, row in constraints_df.iterrows():
    constraints.append(
        Constraint(
            name=row['ConstraintName'],
            setback=row[setback_field_name],
            feature_class=row['Data Source'],
            where_clause=row['where'],
            constraint_class=row['Constraint Class']
        )
    )
    

# Check that data loaded correctly
for c in constraints[:3]:
    print(c.attributes())
    print('')

{'name': 'NumberedHighwayRow', 'setback': 40.0, 'feature_class': '.\\Constraints\\Constraints.gdb\\Highway_ROW', 'where_clause': '', 'constraint_class': 'NoGo'}

{'name': 'CountyRoadsRow', 'setback': 100.0, 'feature_class': '.\\Constraints\\Constraints.gdb\\Road_ROW', 'where_clause': '', 'constraint_class': 'Mitigate'}

{'name': 'Residence', 'setback': 1000.0, 'feature_class': '.\\Constraints\\Constraints.gdb\\Receptors_ResidenceLocations', 'where_clause': "Status = 'Occupied'", 'constraint_class': 'NoGo'}



In [6]:
# All constraint attributes. None FC paths filled in by ConstraintsModel as processing is done
constraints[0].__dict__

{'name': 'NumberedHighwayRow',
 'setback': 40.0,
 'feature_class': '.\\Constraints\\Constraints.gdb\\Highway_ROW',
 'where_clause': '',
 'constraint_class': 'NoGo',
 'clip_fc': None,
 'project_fc': None,
 'spatialjoin_fc': None,
 'constraint_fc': None,
 'constraint_shp': None,
 'constraint_setback_fc': None}

__Initialize ConstraintsModel__

In [9]:
model = ConstraintsModel(
    sites_fc=r'.\Constraints.gdb\Project_Area_Buffer_1000',
    constraints=constraints,
    output_folder=os.path.join(working_folder, 'Constraints'),
    output_epsg=26912,
    overwrite_outputs=False
)

__Execute process_constraints method to apply buffers and sort constraints into gdbs__<br />
__export_shp exports results in folders matching gdb output__

In [ ]:
# model.process_constraints()
# model.export_shp()

__Output Folder stucutre__<br /><br />
├───Constraints<br />
│   ├───gdb<br />
│   │   ├───BeAware.gdb<br />
│   │   ├───Minimize.gdb<br />
│   │   └───Nogo.gdb<br />
│   ├───shp<br />
│   │   ├───BeAware<br />
│   │   ├───Minimize<br />
│   │   └───Nogo<br />
│   └───working<br />
│       ├───Clip.gdb<br />
│       ├───Dissolve.gdb<br />
│       ├───Merge.gdb<br />
│       └───Project.gdb<br />